In [1]:
import os
os.chdir("/home/ubuntu/llm/llm-rag-chatbot")


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import (
    create_openai_functions_agent,
    Tool,
    AgentExecutor
)
from langchain import hub
from chatbot_api.src.chains.hospital_review_chain import reviews_vector_chain
from chatbot_api.src.chains.hospital_cypher_chain import hospital_cypher_chain
from chatbot_api.src.tools.wait_times import (
    get_current_wait_times,
    get_most_available_hospital
)

HOSPITAL_AGENT_MODEL = os.getenv("HOSPITAL_AGENT_MODEL")

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

tools  = [
    Tool(
        name="Experiences",
        func=reviews_vector_chain.invoke,
        description="""Useful when you need to answer questions about patient experiences, feelings, or any other qualitative question that could be answered about a patient using semantic search. 
        Not useful for answering objective questions that involve counting, percentages, aggregations, or listing facts. 
        Use the entire prompt as input to the tool. 
        For instance, if the prompt is "Are patients satisfied with their care?", the input should be "Are patients satisfied with their care?".""",
    ),
    Tool(
        name="Graph",
        func=hospital_cypher_chain.invoke,
        description="""Useful for answering questions about patients, physicians, hospitals, insurance payers, patient review statistics, and hospital visit details. 
        Use the entire prompt as input to the tool. 
        For instance, if the prompt is "How many visits have there been?", the input should be "How many visits have there been?".""",
    ),
    Tool(
        name="Waits",
        func=get_current_wait_times,
        description="""Use when asked about current wait times at a specific hospital. 
        This tool can only get the current wait time at a hospital and does not have any information about aggregate or historical wait times. 
        Do not pass the word "hospital" as input, only the hospital name itself. 
        For example, if the prompt is "What is the current wait time at Jordan Inc Hospital?", the input should be "Jordan Inc".""",
    ),
    Tool(
        name="Availability",
        func=get_most_available_hospital,
        description="""Use when you need to find out which hospital has the shortest wait time. 
        This tool does not have any information about aggregate or historical wait times. 
        This tool returns a dictionary with the hospital name as the key and the wait time in minutes as the value.""",
    ),
]

chat_model = ChatOpenAI(
    model=HOSPITAL_AGENT_MODEL,
    temperature=0
)

hospital_rag_agent = create_openai_functions_agent(
    llm=chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

hospital_rag_agent_executor = AgentExecutor(
    agent=hospital_rag_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True
)


In [4]:
response = hospital_rag_agent_executor.invoke(
    {"input": "What is the wait time at Wallace-Hamilton?"}
)



> Entering new AgentExecutor chain...

Invoking: `Waits` with `Wallace-Hamilton`


1 hours 45 minutesThe current wait time at Wallace-Hamilton is 1 hour and 45 minutes.

> Finished chain.


In [5]:
response.get("output")

'The current wait time at Wallace-Hamilton is 1 hour and 45 minutes.'

In [6]:
response = hospital_rag_agent_executor.invoke(
    {"input": "Which hospital has the shortest wait time?"}
)



> Entering new AgentExecutor chain...

Invoking: `Availability` with `shortest wait time`


{'wheeler, bryant and johns': 31}The hospital with the shortest wait time is Wheeler, Bryant and Johns, with a wait time of 31 minutes.

> Finished chain.


In [7]:
response.get("output")

'The hospital with the shortest wait time is Wheeler, Bryant and Johns, with a wait time of 31 minutes.'

In [8]:
response = hospital_rag_agent_executor.invoke(
    {
        "input": (
            "What have pateients said about their "
            "quality of rest during their stay?"
        )
    }
)



> Entering new AgentExecutor chain...

Invoking: `Experiences` with `What have patients said about their quality of rest during their stay?`


{'query': 'What have patients said about their quality of rest during their stay?', 'result': "Patients have mentioned that the quality of rest during their stay at different hospitals was affected by various factors. Monique Foster at Pearson LLC found it challenging to get a good night's sleep due to constant interruptions for routine checks. Heather Lewis at Brown-Golden mentioned that the constant beeping of machines in the ward was bothersome and made it difficult to get quality rest. Ashley Solis, also at Brown-Golden, highlighted that the noise level in shared rooms made it difficult to get a good night's sleep."}Patients have mentioned that the quality of rest during their stay at different hospitals was affected by various factors. For example, Monique Foster at Pearson LLC found it challenging to get a good night's sleep due to const

In [9]:
response.get("output")

"Patients have mentioned that the quality of rest during their stay at different hospitals was affected by various factors. For example, Monique Foster at Pearson LLC found it challenging to get a good night's sleep due to constant interruptions for routine checks. Heather Lewis at Brown-Golden mentioned that the constant beeping of machines in the ward was bothersome and made it difficult to get quality rest. Ashley Solis, also at Brown-Golden, highlighted that the noise level in shared rooms made it difficult to get a good night's sleep."

In [10]:
response = hospital_rag_agent_executor.invoke(
    {
        "input": (
            "Which physician has treated the "
            "most patients covered by Cigna?"
        )
    }
)



> Entering new AgentExecutor chain...

Invoking: `Grpah` with `Which physician has treated the most patients covered by Cigna?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (phy:Physician)-[:TREATS]->(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Cigna'
WITH phy, COUNT(DISTINCT v) AS patient_count
RETURN phy.name AS physician_name, patient_count
ORDER BY patient_count DESC
LIMIT 1
Full Context:
[{'physician_name': 'Kayla Lawson', 'patient_count': 10}]

> Finished chain.
{'query': 'Which physician has treated the most patients covered by Cigna?', 'result': 'The physician who has treated the most patients covered by Cigna is Kayla Lawson, with a patient count of 10.'}The physician who has treated the most patients covered by Cigna is Kayla Lawson, with a patient count of 10.

> Finished chain.


In [11]:
response.get("output")

'The physician who has treated the most patients covered by Cigna is Kayla Lawson, with a patient count of 10.'

In [14]:
response = hospital_rag_agent_executor.invoke(
    {"input": "Show me reviews written by patient 7674."}
)



> Entering new AgentExecutor chain...

Invoking: `Experiences` with `Show me reviews written by patient 7674.`


{'query': 'Show me reviews written by patient 7674.', 'result': 'I\'m sorry, but there is no review provided by a patient with the identifier "7674" in the context given. If you have any other questions or need information, feel free to ask.'}I'm sorry, but there is no review provided by a patient with the identifier "7674" in the context given. If you have any other questions or need information, feel free to ask.

> Finished chain.


In [17]:
response = hospital_rag_agent_executor.invoke(
    {"input": (
        "Query the graph database to show me "
        "the reviews written by patient 7674"
    )}
)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Show me the reviews written by patient 7674`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {id: 7674})-[:HAS]->(v:Visit)-[:WRITES]->(r:Review)
RETURN r.text AS review_written

Full Context:
[{'review_written': 'The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated.'}]

> Finished chain.
{'query': 'Show me the reviews written by patient 7674', 'result': 'Patient 7674 wrote the following review: "The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."'}Patient 7674 wrote the following review: "The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."

> Finished chain.


In [18]:
response.get("output")

'Patient 7674 wrote the following review: "The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."'